# ML-based crosstalk-free sensor matrix

In the resistive sensor network configuration (row and column scanning), each node can be considered as an element with variable resistance. Ideally, this intersection point should be measured individually by injecting a current to calibrate the measured voltage with the change of pressure applied on the sensor. However, other sensor on the same row and column will distribute the current and leads to the occurance of coupled current, in other words, crosstalk. 

Crosstalk often occurs in sensor network as a systematic noise instead of random noise. The measured resistive sensor network will reflect on the unreal pressure heatmap due to the crosstalk issue. To improve this, a ML based algorithm is proposed to reverse the measured sensor matrix with crosstalk back to the crosstalk-free sensor matrix.

As the first step, we will generate dataset purly based on simulated calculations. Supervised learning will be appllied to find out the hidden relationship between crosstalk sensor matrix and crosstalk-free one. The crosstalk-free sensor matrix will be generated randomly as the label dataset and forward-calculations will be performed to simulate the crosstalk and generate the input dataset. 

## Import Libs

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Forward Operation

In [119]:
class ResistorGridNetwork:
    def __init__(self, rows, cols, R_min=100, R_max=1000, Rseries=300, Vcc=3.3, uniform=True):
        """
        rows, cols : size of matrix
        R_min, R_max : range of resistors in the network
        Rseries, Vcc : external resistor in series, voltage source
        """
        
        self.rows = rows
        self.cols = cols
        self.R_min = R_min
        self.R_max = R_max
        self.Rseries = Rseries
        self.Vcc = Vcc
        self.uniform = uniform

        self.R_matrix_raw = self.generate_random_resistance_matrix()

        self.num_nodes = self.rows + self.cols

        self.G_base = self.build_conductance_matrix()
   
    def generate_random_resistance_matrix(self, R_min=None, R_max=None, uniform=None):
        """
        Generate a random resistance matrix with shape (rows, cols) for MAT or non-uniform insole pattern.
        """
        shape = (self.rows, self.cols)

        # If R_min and R_max are not provided, use the default values
        R_min_used = R_min if R_min is not None else self.R_min
        R_max_used = R_max if R_max is not None else self.R_max
        # MAT or Insole pattern
        uniform_used = uniform if uniform is not None else self.uniform

        if uniform_used:
            R_matrix_raw = np.random.uniform(R_min_used, R_max_used, size=shape)
        else:
            pass # implement insole pattern here
        return R_matrix_raw

    def build_conductance_matrix(self):

        """
        Implement the following algorithm to build the conductance matrix G_base:
        1. Initialize a (rows+cols) x (rows+cols) matrix G_base with zeros.
        2. For each element R_matrix_raw[i,j], treat it as a resistor between row node i and col node (rows + j).
        3. Update the conductance matrix G_base:
            G[u,u] += g_ij
            G[v,v] += g_ij
            G[u,v] -= g_ij
            G[v,u] -= g_ij
        where:
            u = i        (row node)
            v = rows + j (col node)
        """
        
        R = self.rows
        C = self.cols
        N = self.num_nodes
        
        G = np.zeros((N, N))
        
        for i in range(R):
            for j in range(C):
                r_ij = self.R_matrix_raw[i, j]
                
                # Row node number
                row_node = i
                # Column node number
                col_node = R + j
                
                g_ij = 1.0 / r_ij  
                G[row_node, row_node] += g_ij
                G[col_node, col_node] += g_ij
                G[row_node, col_node] -= g_ij
                G[col_node, row_node] -= g_ij
        
        return G
    
    def measure_single(self, row_idx, col_idx):
        """
        Measure the voltage difference between the two nodes connected by the resistor at (row_idx, col_idx).
        1. Build a new conductance matrix G by copying G_base.
        2. Modify G and b to model the voltage source and the ground.
        3. Solve Gv = b to get the voltage vector v.
        """
        G = self.G_base.copy()
        b = np.zeros(self.num_nodes) 
        
        # 1) row_idx connected to voltage source: G[row_idx,row_idx] += 1/Rseries, b[row_idx] += Vcc/Rseries
        G[row_idx, row_idx] += 1.0 / self.Rseries
        b[row_idx] += self.Vcc / self.Rseries
        
        # 2) col_idx to GND
        gnode = self.rows + col_idx
        for c in range(self.num_nodes):
            if c != gnode:
                G[gnode, c] = 0
                G[c, gnode] = 0
        G[gnode, gnode] = 1
        b[gnode] = 0
        
        # solve Gv = b
        v_sol = np.linalg.solve(G, b)
        return v_sol
    
    def measure_crosstalk_all(self):
        """
        Measure the crosstalk for all resistors in the network.
        """
        R = self.rows
        C = self.cols
        voltage_matrix_crosstalk = np.zeros((R, C))
        
        for i in range(R):
            for j in range(C):
                v = self.measure_single(i, j)
                row_node = i
                col_node = R + j
                diff = v[row_node] - v[col_node]
                voltage_matrix_crosstalk[i,j] = diff
        return voltage_matrix_crosstalk
    

    def measure_ideal_all(self):
        # Calculate the ideal voltage matrix
        voltage_matrix_ideal = np.zeros((self.R_matrix_raw.shape[0], self.R_matrix_raw.shape[1]))
        for i in range(self.R_matrix_raw.shape[0]):
            for j in range(self.R_matrix_raw.shape[1]):
                voltage_matrix_ideal[i][j] = self.Vcc * self.R_matrix_raw[i][j] / (self.Rseries + self.R_matrix_raw[i][j])
        return voltage_matrix_ideal

## Generate training dataset

In [121]:
if __name__ == "__main__":
    network = ResistorGridNetwork(3,3, R_min=1000, R_max=1000, Rseries=1000, Vcc=3.3, uniform=True)

    print("\n[R_matrix_raw], shape=", network.R_matrix_raw.shape)
    print(network.R_matrix_raw) 

    v_sol = network.measure_single(row_idx=0, col_idx=0)
    
    crosstalk_result = network.measure_crosstalk_all()
    print("\n [crosstalk_result], shape=", crosstalk_result.shape)
    print(crosstalk_result) # Input dataset for the ML model
    
    ideal_result = network.measure_ideal_all()
    print("\n [ideal_result], shape=", ideal_result.shape)
    print(ideal_result) # Label dataset for the ML model
    
    


[R_matrix_raw], shape= (3, 3)
[[1000. 1000. 1000.]
 [1000. 1000. 1000.]
 [1000. 1000. 1000.]]

 [crosstalk_result], shape= (3, 3)
[[1.17857143 1.17857143 1.17857143]
 [1.17857143 1.17857143 1.17857143]
 [1.17857143 1.17857143 1.17857143]]

 [ideal_result], shape= (3, 3)
[[1.65 1.65 1.65]
 [1.65 1.65 1.65]
 [1.65 1.65 1.65]]
